<a href="https://colab.research.google.com/github/cheong0522/KBSC/blob/main/%EB%B0%B1%EC%97%851.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import math

In [6]:
!pip install soynlp emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 31.7 MB/s 
     |████████████████████████████████| 197 kB 51.5 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=e5c618c20948d151ebf6259def2eb6a100993d183cbf966599af598ac43c01b8
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 34.0 MB/s 
     |████████████████████████████████| 120 kB 75.3 MB/s 
     |████████████████████████████████| 6.6 MB 57.6 MB/s 


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AdamW 
import numpy as np
from tqdm.notebook import tqdm

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [11]:
device = torch.device("cuda")

In [12]:
test_data = pd.read_excel('/content/drive/MyDrive/KBSC_data.xlsx')

In [13]:

test_data.loc[(test_data['Emotion'] == "행복"), 'Emotion'] = 0 
test_data.loc[(test_data['Emotion'] == "슬픔"), 'Emotion'] = 1  
test_data.loc[(test_data['Emotion'] == "분노"), 'Emotion'] = 2  
test_data.loc[(test_data['Emotion'] == "불안"), 'Emotion'] = 3  
test_data.loc[(test_data['Emotion'] == "중립"), 'Emotion'] = 4  

train_dataset = []
for sen, label in zip(test_data['Sentence'], test_data['Emotion']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(label))

  train_dataset.append(data_train)

class TrainDataset(Dataset):
  
  def __init__(self, dataset):
    self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

    self.sentences = [str([i[0]]) for i in dataset]
    self.labels = [np.int32(i[1]) for i in dataset]

  def __len__(self):
    return (len(self.labels))
  
  def __getitem__(self, i):
    text = self.sentences[i]
    y = self.labels[i]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

train_dataset = TrainDataset(train_dataset)


Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
from torch import nn

model = AutoModel.from_pretrained("beomi/KcELECTRA-base", num_labels=363)
model = model.to(device)

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
!pip install emoji==1.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 10.0 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=82000514483a1c56ef3b1fa8448732561ed75eba1aa3ebad135e96b5aa35d0a0
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.0.0
    Uninstalling emoji-2.0.0:
      Successfully uninstalled emoji-2.0.0


In [16]:
import re
import emoji
from soynlp.normalizer import repeat_normalize


In [17]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x):
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x


In [18]:
batch_size = 32
epochs = 2

optimizer = AdamW(model.parameters(), lr=3e-5)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=5, shuffle=True)

losses = []
accuracies = []

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

loss_fn = nn.CrossEntropyLoss()

for i in range(epochs):
  train_acc = 0.0
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_dataloader):
    optimizer.zero_grad()
    y_batch = y_batch.long().to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    y_pred = y_pred[:, -1, :]
    loss = loss_fn(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    train_acc += calc_accuracy(y_pred, y_batch)
    total += len(y_batch)

    batches += 1
    if batches % 50 == 0:
      print("epoch {} loss {} train acc {}".format(i+1, loss.data.cpu().numpy(), train_acc / (batches+1)))
  print("epoch {} loss {} train acc {}".format(i+1, loss.data.cpu().numpy(), train_acc / (batches+1)))
  model.eval()

torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/model.pt")

def predict(sentence):
    data = [sentence, '0']
    dataset_another = [data]
    logits = 0
    another_test = TrainDataset(dataset_another)
    test_dataloader = torch.utils.data.DataLoader(another_test)

    model.eval()

    for input_ids_batch, attention_masks_batch, y_batch in test_dataloader:
        y_batch = y_batch.long().to(device)
        out = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
        out = out[:, -1, :]

        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()
            logits = np.argmax(logits)
    return logits

predict("일기 내용")

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/853 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2306: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2306: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

epoch 1 loss 3.879049301147461 train acc 0.15441176470588236
epoch 1 loss 3.2186241149902344 train acc 0.2431930693069307
epoch 1 loss 3.0471506118774414 train acc 0.294908940397351
epoch 1 loss 2.7706573009490967 train acc 0.3347325870646766
epoch 1 loss 2.1709542274475098 train acc 0.367156374501992
epoch 1 loss 1.8673547506332397 train acc 0.39680232558139533
epoch 1 loss 2.2193076610565186 train acc 0.41978276353276356
epoch 1 loss 2.192192554473877 train acc 0.43718827930174564
epoch 1 loss 2.608844041824341 train acc 0.44983370288248337
epoch 1 loss 1.7115329504013062 train acc 0.4653817365269461
epoch 1 loss 2.9724695682525635 train acc 0.4791288566243194
epoch 1 loss 1.6636515855789185 train acc 0.48954866888519133
epoch 1 loss 1.839147925376892 train acc 0.4999519969278034
epoch 1 loss 2.3225820064544678 train acc 0.5102977888730386
epoch 1 loss 2.664865255355835 train acc 0.5191827563249002
epoch 1 loss 1.5905040502548218 train acc 0.5284410112359551
epoch 1 loss 1.3387715816

  0%|          | 0/853 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2306: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2306: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

epoch 2 loss 1.4522075653076172 train acc 0.6856617647058824
epoch 2 loss 1.4823130369186401 train acc 0.7097772277227723
epoch 2 loss 1.8861278295516968 train acc 0.71875
epoch 2 loss 1.5914877653121948 train acc 0.7223258706467661
epoch 2 loss 1.3493072986602783 train acc 0.7254731075697212
epoch 2 loss 2.114102363586426 train acc 0.7316237541528239
epoch 2 loss 1.7348870038986206 train acc 0.7346866096866097
epoch 2 loss 1.5627015829086304 train acc 0.7325436408977556
epoch 2 loss 1.482925295829773 train acc 0.7337167405764967
epoch 2 loss 1.4908020496368408 train acc 0.7369011976047904
epoch 2 loss 1.5943048000335693 train acc 0.7374659709618875
epoch 2 loss 1.4680695533752441 train acc 0.7372088186356073
epoch 2 loss 1.216148018836975 train acc 0.7373751920122887
epoch 2 loss 1.9619834423065186 train acc 0.7376069900142653
epoch 2 loss 1.975771427154541 train acc 0.7386401464713716
epoch 2 loss 1.5729085206985474 train acc 0.7397393882646691
epoch 2 loss 1.5786125659942627 train a

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2306: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


4

In [19]:
emotion = predict("암거나 문장 넣어서 테스트 해봐유")

import random
num = random.randint(1,31)

In [26]:
print(num)

if emotion == 0 :
  happy_music_data = pd.read_excel('/content/drive/MyDrive/happy_music_data.xlsx')
  print(happy_music_data.loc[num])
elif emotion == 1 :
  sad_music_data = pd.read_excel('/content/drive/MyDrive/sad_music_data.xlsx')
  print(sad_music_data.loc[num])
elif emotion == 2 :
  anger_music_data = pd.read_excel('/content/drive/MyDrive/anger_music_data.xlsx')
  print(anger_music_data.loc[num])
elif emotion == 3 :
  nervous_music_data = pd.read_excel('/content/drive/MyDrive/nervous_music_data.xlsx')
  print(nervous_music_data.loc[num])
elif emotion == 4 :
  neutrality_music_data = pd.read_excel('/content/drive/MyDrive/neutrality_music_data.xlsx')
  print(neutrality_music_data.loc[num])


4
IVE (아이브)     IVE (아이브)
After LIKE    LOVE DIVE
Name: 4, dtype: object


In [31]:
# Save model.
!pip install pyyaml h5py 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.8.2


In [35]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [36]:
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [37]:
!mkdir -p saved_model
model.save('saved_model/my_model') 

In [38]:
# Convert Keras model to TF Lite format.

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [39]:
# Save the TF Lite model as file
f = open('my_model.tflite', "wb")
f.write(tflite_model)
f.close()

In [40]:
# Read lite model for tensorflow
# tflite_model = None

tflite_path = "my_model.tflite"
with open(tflite_path, 'rb') as f:
  tflite_model = f.read()

In [41]:
# Run inference with TensorFlow Lite
"""
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
interpreter.set_tensor(interpreter.get_input_details()[0]["index"], emotion)
interpreter.invoke()
output = interpreter.tensor(interpreter.get_output_details()[0]["index"])()[0]
"""

ValueError: ignored